In [13]:
# Import & Set up
import graph
import random
import patrols
import pickle
import plotter
import stats

g = graph.Graph()
g.loadGraph("./out/saved_graph_races.pkl")

In [14]:
### LAPD REAL ARREST COUNTS###
# Plot LAPD Arrest Record Counts

# Plot All of LA
real_arrest_counts = {ids:len(n.pd_reports) for ids, n in g.nodes.items()}
real_arrest_mean = stats.sim_mean(real_arrest_counts)
real_arrest_stddev = stats.sim_stddev(real_arrest_counts)

plotter.imshow_plot(real_arrest_counts, "./out/graphics/LAPD_arrest_count/","All_LA", 
                    "LAPD Arrest Record Count",real_arrest_mean, real_arrest_stddev, False)

# Plot individual Precincts
for pID in range(1,22):
    p_arr_cnt = {ids:len(n.pd_reports) for ids, n in g.precincts[pID].items()}
    p_arr_mean = stats.sim_mean(p_arr_cnt)
    p_arr_stddev = stats.sim_stddev(p_arr_cnt)
    
    plotter.imshow_plot(p_arr_cnt, "./out/graphics/LAPD_arrest_count/", f"Prec {pID}", 
                        f"LAPD Prec.{pID} Arrest Record Count", p_arr_mean, p_arr_stddev, False)
    
### FINISHED PLOTTING LAPD REAL ARREST COUNTS###

In [15]:
### UNIFORM ARREST PROBABILITY

# Create list of all LA nodes, & list for each precinct
sim_list = []
p_sim_list = {i:[] for i in range(1,22)}

uniform_arrest_rate = {node:0.35 for node in g.nodes.keys()}

for t in range(10):
    # run 10 trials
    all_nodes = {}

    for pID in range(1,22):
        while(True):
            start = random.choice(list(g.precincts[pID].values()))
            try:
                path, arrests = patrols.random_walk_precinct(45000, g, start, uniform_arrest_rate)
            except:
                continue
            else:
                p_sim_list[pID].append(arrests)
                all_nodes.update(arrests)
                break

    sim_list.append(all_nodes)

# Simulations complete: 
# Analysis follows
## Analysis for All LA
la_store = stats.result(sim_list)
la_cnt = {ids:n['mean'] for ids,n in la_store.items()}

la_mean = stats.sim_mean(la_cnt)
la_stddev = stats.sim_stddev(la_cnt)

la_similarity = stats.similarity_score(la_store, real_arrest_counts)

plotter.imshow_plot(la_cnt, "./out/graphics/Uniform_Dist/","All_LA", 
                    "Simulated Arrest Count",la_mean, la_stddev, la_similarity)

la_ans, (c1,c2,c3) = stats.compare_sims(la_store, real_arrest_counts)
conf = {ids:sum(tup) for ids,tup in la_ans.items()}

plotter.imshow_plot(conf, "./out/graphics/Uniform_Dist/","All_LA_conf", 
                    "Nodes within CI",False, False, False)

## Analysis per precinct
for pid in range(1,22):
    siml = p_sim_list[pid]
    real_cnt = {ids:len(n.pd_reports) for ids,n in g.precincts[pid].items()}
    
    p_store = stats.result(siml)
    p_cnt = {ids:n['mean'] for ids,n in p_store.items()}
    
    p_mean = stats.sim_mean(p_cnt)
    p_stddev = stats.sim_stddev(p_cnt)
    
    p_similarity = stats.similarity_score(p_store, real_cnt)
    
    plotter.imshow_plot(p_cnt, "./out/graphics/Uniform_Dist/",f"Prec{pid}", 
                    f"Prec. {pid} Simulated Arrest Count",p_mean, p_stddev, p_similarity)
    
    p_ans, (c1,c2,c3) = stats.compare_sims(p_store, real_cnt)
    conf = {ids:sum(tup) for ids,tup in p_ans.items()}
    
    plotter.imshow_plot(conf, "./out/graphics/Uniform_Dist/",f"Prec{pid}_conf", 
                    f"Prec. {pid} Nodes within CI",False, False, p_similarity)

In [17]:
### POPULATION BASED ARREST PROBABILITY
tract_nodes = []
for nid, node in g.nodes.items():
    if(node.population == 0):
        node.population =  {"POP": 0, "LATPOP_D": 0,"WHIPOP_D": 0, "BLAPOP_D": 0, "AMIPOP_D": 0,
                            "ASIPOP_D": 0, "HPIPOP_D": 0, "OTHPOP_D": 0, "MMRPOP_D": 0,}
        
races = ["POP", "LATPOP_D", "WHIPOP_D", "BLAPOP_D", "AMIPOP_D", "ASIPOP_D", "HPIPOP_D", "OTHPOP_D", "MMRPOP_D"]
sim_list_by_race = {race:[] for race in races}


for race_id in races:
    min_pop = min([node.population[race_id] for nid, node in g.nodes.items()])
    max_pop = max([node.population[race_id] for nid, node in g.nodes.items()])
    # p = (cur - min) / (max - min)
    arrest_rate = {node_id:((n.population[race_id] - min_pop) / (max_pop - min_pop)) for node_id, n in g.nodes.items()}
    
    for t in range(10):
        all_nodes = {}
        for pID in range(1,22):
            while(True):
                start = random.choice(list(g.precincts[pID].values()))
                try:
                    path, arrests = patrols.random_walk_precinct(50000, g, start, arrest_rate)
                except:
                    continue
                else:
                    all_nodes.update(arrests)
                    break
            
        sim_list_by_race[race_id].append(all_nodes)
    

for rID, r_sim_list in sim_list_by_race.items():
    r_store =  stats.result(r_sim_list)
    r_cnt = {ids:n['mean'] for ids, n in r_store.items()}
    
    r_mean = stats.sim_mean(r_cnt)
    r_stddev = stats.sim_stddev(r_cnt)
    
    r_similarity = stats.similarity_score(r_store, real_arrest_counts)
    
    ans, (c1,c2,c3) = stats.compare_sims(r_store, real_arrest_counts)
    
    conf = {ids:sum(tup) for ids, tup in ans.items()}
    
    plotter.imshow_plot(r_cnt, "./out/graphics/Racial_Dist/",f"{rID}", 
                    f"{rID} Weighted Simulated Arrest Count",r_mean, r_stddev, r_similarity)
    
    plotter.imshow_plot(conf, "./out/graphics/Racial_Dist/",f"{rID}_conf", 
                    f"{rID} Nodes within CI",False, False, r_similarity)